In [1]:
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.automl import H2OAutoML
from h2o.estimators import H2ORandomForestEstimator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_data = pd.read_csv("/kaggle/input/duomenys/train_data.csv")
test_data= pd.read_csv("/kaggle/input/projektas/test_data.csv")

train_data['credit_score'] = train_data['credit_score'].fillna('unknown')
test_data['credit_score'] = test_data['credit_score'].fillna('unknown')

train_data["months_since_last_delinquent"] = train_data["months_since_last_delinquent"].fillna(value = 300)
test_data["months_since_last_delinquent"] = test_data["months_since_last_delinquent"].fillna(value = 300)

h2o.init()

train_data = h2o.H2OFrame(train_data)

test_data= h2o.H2OFrame(test_data)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.16" 2022-07-19; OpenJDK Runtime Environment (build 11.0.16+8-post-Ubuntu-0ubuntu120.04); OpenJDK 64-Bit Server VM (build 11.0.16+8-post-Ubuntu-0ubuntu120.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpcf2g1tpd
  JVM stdout: /tmp/tmpcf2g1tpd/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpcf2g1tpd/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.1
H2O_cluster_version_age:,1 month and 27 days
H2O_cluster_name:,H2O_from_python_unknownUser_i3o9ku
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.500 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [3]:
train_data = train_data.drop('id', axis=1)

train,test,valid = train_data.split_frame(ratios=[.7, .15], seed = 1)

x = test.columns
y = 'y'
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [4]:
# Run AutoML for 20 base models
# "DRF", "GLM", "XGBoost", "GBM", "DeepLearning", "StackedEnsemble"
aml = H2OAutoML(max_models=1, seed=1, exclude_algos = ["DRF", "GLM", "GBM", "DeepLearning", "StackedEnsemble"])
#max_runtime_secs=3600*3
#aml = H2OAutoML(max_models=25, seed=1, max_runtime_secs=3600*0.5)
aml.train(x=x, y=y, training_frame=train, validation_frame=valid)

AutoML progress: |
20:01:02.287: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_1_AutoML_1_20221115_200102


Model Summary: 
    number_of_trees
--  -----------------
    83

ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.1062636336990459
RMSE: 0.32598103272897017
LogLoss: 0.33939356875986615
Mean Per-Class Error: 0.20528342776069594
AUC: 0.8906264014514278
AUCPR: 0.7824226291817413
Gini: 0.7812528029028556

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.31697588598322024
       0            1            Error    Rate
-----  -----------  -----------  -------  ---------------------
0      4.74684e+06  558693       0.1053   (558693.0/5305538.0)
1      517283       1.17726e+06  0.3053   (517283.0/1694548.0)
Total  5.26413e+06  1.73596e+06  0.1537   (1075976.0/7000086.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value        idx
---------------------------  -----------  -----------  -----
max f1                       0.316976     0.686351     205
max f2                       0.194456     0.750487     276
max f0point5                 0.46124      0.741316     142
max accuracy                 0.407375     0.859237     164
max precision                0.999913     1            0
max recall                   0.00825412   1            398
max specificity              0.999913     1            0
max absolute_mcc             0.373938     0.591551     178
max min_per_class_accuracy   0.244061     0.801295     244
max mean_per_class_accuracy  0.258206     0.803219     236
max tns                      0.999913     5.30554e+06  0
max fns                      0.999913     1.37404e+06  0
max fps                      1.48719e-05  5.30554e+06  399
max tps                      0.00825412   1.69455e+06  398
max tnr                      0.999913     1            0
max fnr                      0.999913     0.810861     0
max fpr                      1.48719e-05  1            399
max tpr                      0.00825412   1            398

Gains/Lift Table: Avg response rate: 24.21 %, avg score: 24.21 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.010055                    0.99997            4.13095    4.13095            1                0.99998      1                           0.99998             0.0415367       0.0415367                  313.095   313.095            0.0415367
2        0.0200335                   0.999951           4.13095    4.13095            1                0.999961     1                           0.99997             0.0412204       0.0827572                  313.095   313.095            0.0827572
3        0.0300231                   0.999922           4.13095    4.13095            1                0.999938     1                           0.99996             0.0412665       0.124024                   313.095   313.095            0.124024
4        0.0400027                   0.999848           4.13095    4.13095            1                0.999893     1                           0.999943            0.0412252       0.165249                   313.095   313.095            0.165249
5        0.0500001                   0.848624           4.08496    4.12175            0.988869         0.954022     0.997774                    0.990761            0.0408392       0.206088                   308.496   312.175            0.205941
6        0.1                         0.57392            3.48144    3.8016             0.84277          0.68115      0.920272                    0.835956    

In [5]:
model = aml.leader

In [6]:
train_perf = model.model_performance(train=True)
train_perf

ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.1062636336990459
RMSE: 0.32598103272897017
LogLoss: 0.33939356875986615
Mean Per-Class Error: 0.20528342776069594
AUC: 0.8906264014514278
AUCPR: 0.7824226291817413
Gini: 0.7812528029028556

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.31697588598322024
       0            1            Error    Rate
-----  -----------  -----------  -------  ---------------------
0      4.74684e+06  558693       0.1053   (558693.0/5305538.0)
1      517283       1.17726e+06  0.3053   (517283.0/1694548.0)
Total  5.26413e+06  1.73596e+06  0.1537   (1075976.0/7000086.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value        idx
---------------------------  -----------  -----------  -----
max f1                       0.316976     0.686351     205
max f2                       0.194456     0.750487     276
max f0point5                 0.46124      0.741316     142
max accuracy                 0.407375     0.859237     164
max precision                0.999913     1            0
max recall                   0.00825412   1            398
max specificity              0.999913     1            0
max absolute_mcc             0.373938     0.591551     178
max min_per_class_accuracy   0.244061     0.801295     244
max mean_per_class_accuracy  0.258206     0.803219     236
max tns                      0.999913     5.30554e+06  0
max fns                      0.999913     1.37404e+06  0
max fps                      1.48719e-05  5.30554e+06  399
max tps                      0.00825412   1.69455e+06  398
max tnr                      0.999913     1            0
max fnr                      0.999913     0.810861     0
max fpr                      1.48719e-05  1            399
max tpr                      0.00825412   1            398

Gains/Lift Table: Avg response rate: 24.21 %, avg score: 24.21 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.010055                    0.99997            4.13095    4.13095            1                0.99998      1                           0.99998             0.0415367       0.0415367                  313.095   313.095            0.0415367
2        0.0200335                   0.999951           4.13095    4.13095            1                0.999961     1                           0.99997             0.0412204       0.0827572                  313.095   313.095            0.0827572
3        0.0300231                   0.999922           4.13095    4.13095            1                0.999938     1                           0.99996             0.0412665       0.124024                   313.095   313.095            0.124024
4        0.0400027                   0.999848           4.13095    4.13095            1                0.999893     1                           0.999943            0.0412252       0.165249                   313.095   313.095            0.165249
5        0.0500001                   0.848624           4.08496    4.12175            0.988869         0.954022     0.997774                    0.990761            0.0408392       0.206088                   308.496   312.175            0.205941
6        0.1                         0.57392            3.48144    3.8016             0.84277          0.68115      0.920272                    0.835956            0.174072        0.38016                    248.144   280.16             0.369641
7        0.15                        0.450603           2.70378    3.43566         

In [7]:
valid_perf = model.model_performance(valid=True)
valid_perf

ModelMetricsBinomial: xgboost
** Reported on validation data. **

MSE: 0.12009460996108483
RMSE: 0.3465466923245478
LogLoss: 0.3771323292521324
Mean Per-Class Error: 0.24554068627630676
AUC: 0.8465534458573445
AUCPR: 0.7114623790555856
Gini: 0.693106891714689

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3058630799501922
       0           1       Error    Rate
-----  ----------  ------  -------  --------------------
0      981909      155059  0.1364   (155059.0/1136968.0)
1      128793      234309  0.3547   (128793.0/363102.0)
Total  1.1107e+06  389368  0.1892   (283852.0/1500070.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value        idx
---------------------------  -----------  -----------  -----
max f1                       0.305863     0.622773     203
max f2                       0.167152     0.705991     289
max f0point5                 0.482423     0.675768     123
max accuracy                 0.451923     0.834611     135
max precision                0.999912     1            0
max recall                   1.46721e-05  1            399
max specificity              0.999912     1            0
max absolute_mcc             0.393728     0.507804     160
max min_per_class_accuracy   0.23158      0.758847     245
max mean_per_class_accuracy  0.257349     0.760282     230
max tns                      0.999912     1.13697e+06  0
max fns                      0.999912     294658       0
max fps                      1.46721e-05  1.13697e+06  399
max tps                      1.46721e-05  363102       399
max tnr                      0.999912     1            0
max fnr                      0.999912     0.811502     0
max fpr                      1.46721e-05  1            399
max tpr                      1.46721e-05  1            399

Gains/Lift Table: Avg response rate: 24.21 %, avg score: 24.11 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100022                   0.99997            4.13126   4.13126            1                0.99998      1                           0.99998             0.0413217       0.0413217                  313.126   313.126            0.0413217
2        0.0200257                   0.999951           4.13126   4.13126            1                0.999961     1                           0.99997             0.0414099       0.0827316                  313.126   313.126            0.0827316
3        0.0300346                   0.999921           4.13126   4.13126            1                0.999937     1                           0.999959            0.0413493       0.124081                   313.126   313.126            0.124081
4        0.0400061                   0.999844           4.13126   4.13126            1                0.999891     1                           0.999942            0.041195        0.165276                   313.126   313.126            0.165276
5        0.0500003                   0.835822           3.91384   4.0878             0.947372         0.948206     0.989481                    0.989601            0.0391157       0.204392                   291.384   308.78             0.203698
6        0.1                         0.558606           2.97659   3.5322             0.720505         0.665121     0.854993                    0.827362            0.148829        0.35322                    197.659   253.22             0.334089
7        0.15                        0.441668           2.29521   3.11987            0.55557          0.49498      0.75518

In [8]:
test_perf = model.model_performance(test_data=test)
test_perf

ModelMetricsBinomial: xgboost
** Reported on test data. **

MSE: 0.12005876655989699
RMSE: 0.3464949733544442
LogLoss: 0.3770088973335095
Mean Per-Class Error: 0.245565635458173
AUC: 0.8470287307166241
AUCPR: 0.7126583379516481
Gini: 0.6940574614332482

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.30806281568609106
       0            1       Error    Rate
-----  -----------  ------  -------  --------------------
0      983361       152726  0.1344   (152726.0/1136087.0)
1      129752       234005  0.3567   (129752.0/363757.0)
Total  1.11311e+06  386731  0.1883   (282478.0/1499844.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value        idx
---------------------------  -----------  -----------  -----
max f1                       0.308063     0.623608     204
max f2                       0.16943      0.706437     289
max f0point5                 0.474948     0.677112     130
max accuracy                 0.448153     0.834683     140
max precision                0.999912     1            0
max recall                   0.00787692   1            398
max specificity              0.999912     1            0
max absolute_mcc             0.401037     0.508515     160
max min_per_class_accuracy   0.232843     0.757992     247
max mean_per_class_accuracy  0.256539     0.761022     233
max tns                      0.999912     1.13609e+06  0
max fns                      0.999912     294684       0
max fps                      1.43028e-05  1.13609e+06  399
max tps                      0.00787692   363757       398
max tnr                      0.999912     1            0
max fnr                      0.999912     0.810112     0
max fpr                      1.43028e-05  1            399
max tpr                      0.00787692   1            398

Gains/Lift Table: Avg response rate: 24.25 %, avg score: 24.16 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100137                   0.99997            4.1232    4.1232             1                0.99998      1                           0.99998             0.0412886       0.0412886                  312.32    312.32             0.0412886
2        0.0200234                   0.999951           4.1232    4.1232             1                0.999961     1                           0.99997             0.0412721       0.0825606                  312.32    312.32             0.0825606
3        0.0300218                   0.999922           4.1232    4.1232             1                0.999938     1                           0.999959            0.0412253       0.123786                   312.32    312.32             0.123786
4        0.0400075                   0.99985            4.1232    4.1232             1                0.999893     1                           0.999943            0.0411731       0.164959                   312.32    312.32             0.164959
5        0.0500005                   0.843375           3.91      4.08059            0.948292         0.955326     0.989666                    0.991026            0.0390728       0.204032                   291       308.059            0.20335
6        0.1                         0.560741           2.98233   3.53146            0.723304         0.668908     0.856486                    0.829968            0.149116        0.353148                   198.233   253.146            0.334201
7        0.15                        0.442811           2.30248   3.1218             0.55842          0.496712     0.757131 

In [9]:
model_path = h2o.save_model(model=model, path="/kaggle/working", force=True)
model_path

'/kaggle/working/XGBoost_1_AutoML_1_20221115_200102'

In [10]:
aml.leader.predict(test)

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


predict,p0,p1
0,0.849165,0.150835
0,0.873254,0.126746
0,0.836991,0.163009
0,0.893095,0.106905
0,0.970991,0.029009
0,0.842829,0.157171
1,0.68028,0.31972
1,0.571493,0.428507
0,0.931306,0.0686944
1,0.535858,0.464142


In [11]:
aml.leaderboard

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
XGBoost_1_AutoML_1_20221115_200102,0.840406,0.382309,0.701316,0.251392,0.349429,0.122101


In [12]:
ids=test_data['id']
predict = model.predict(test_data)

output = ids.cbind(predict['p0'])
output.set_names(['id','y'])
h2o.export_file(output, path = "/kaggle/working/subm1.csv", force = True)

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
Export File progress: |██████████████████████████████████████████████████████████| (done) 100%
